In [11]:
from astropy.io import fits
import numpy as np
import os
import glob
import pandas as pd
from astropy.coordinates import SkyCoord

In [31]:
# Julius function to read target list 

def read_target_list(filename):

    """
    Function to read a target list from a .csv file and add the SkyCoord objects as a new column. \n
    It further renames the columns "TARGET_RA" and "TARGET_DEC" to "RA" and "dec"
 
    Parameters
    ----------

    filename: str
        filename is the path specifying where to get the target list as a .csv file from

    Returns
    -------

    file: pd.DataFrame
        target list containing the .csv files information and the added SkyCoords as a pandas data frame object
    """

    # Read from a file
    print("Reading csv table...")
    file = pd.read_csv(filename)
    
    # Rename columns
    file = file.rename(columns = {"TARGET_RA": "RA", "TARGET_DEC": "dec"})

    # Calculate and add SkyCoords for AltAz calculation later
    print("Adding SkyCoords to it...")
    sky_coords = SkyCoord(file["RA"], file["dec"], unit = "deg")

    file["SKY_COORD"] = sky_coords
    
    #Return the Pandas dataframe (i.e. The unmodified initial target list)
    print("Finished, now returning pandas df")
    return file

# target_list = read_target_list('../Galaxy Selection/PGAL/PGAL_S230619bg.ecsv')

In [38]:
target_list = read_target_list('../Galaxy Selection/PGAL/PGAL_S230619bg.ecsv')
list_of_all_wst_r_band_images = glob.glob("/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_*[0123456789].fits")

Reading csv table...
Adding SkyCoords to it...
Finished, now returning pandas df


In [39]:
# So swarp doesnt check every image

def get_list_of_wst_overlapping_images(ra, dec, list_of_wst_images, distance=35):
    
    goodlist = []
    
    for image in list_of_wst_images:
        # header = fits.read_header(image)
        # check if CRVAL1,2 are less than [distance] arcmin away from [ra,dec]
        # if yes, append to goodlist
        hdul = fits.open(image)
        header = hdul[0].header
        crval1 = header["CRVAL1"] # RA
        crval2 = header["CRVAL2"] # Dec
        delt_ra = (ra - crval1) * 60 # [arcmin]
        delt_dec = (dec- crval2) * 60 # [arcmin]
        if delt_ra< 35 and delt_dec<35: 
            goodlist.append(image)
    return goodlist

def get_output_filename(galaxy):
    for galaxy in target_list:
        ra = galaxy['ra']
        dec = galaxy['dec']
        return f"galaxy_ra{ra}_dec{dec}.fits"


get_list_of_wst_overlapping_images(109.679989646446 , 63.1859641956687, list_of_all_wst_r_band_images, distance = 35)

[]

In [37]:
# os.sys("module load swarp")

for galaxy in target_list:
    list_of_relevant_images = get_list_of_wst_overlapping_images(galaxy['ra'],galaxy['dec'],list_of_all_wst_r_band_images, distance = 35)
    output_filename = get_output_filename(galaxy)
    os.sys("swarp -c swarp.conf -CENTER="+str(galaxy['ra'])+","+galaxy['dec'] + " " + list_of_relevant_images) 
    

TypeError: string indices must be integers

In [36]:
# image = '../../V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_048.fits'